In [1]:
import openai
import pandas as pd
import json
import yaml
from IPython.display import Markdown
import sys
import yaml
from tqdm import tqdm

In [2]:
zz=openai.Model.list()

In [3]:
len(zz['data'])

62

In [4]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,
}

In [5]:
def print_save(string, file='../cmp2.md'):
    print(string)
    with open(file,'w') as f:
        f.write(string)

## Load prompt_template.md

In [6]:
sys.path.append('..')
from utils import *
from functools import partial

In [7]:
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [8]:
prompt_dict().keys()

dict_keys(['Preamble', 'Problem-solver', 'Conversation summarizer', 'Construct Kinetic Hamiltonian (continuum version, single-particle)', 'Construct Kinetic Hamiltonian (continuum version, second-quantized)', 'Construct Kinetic Hamiltonian (lattice version)', 'Define each term in Kinetic Hamiltonian (continuum version)', 'Construct Potential Hamiltonian (continuum version)', 'Define each term in Potential Hamiltonian (continuum version)', 'Construct interaction Hamiltonian (real space, lattice version)', 'Construct interaction Hamiltonian (momentum space)', 'Convert from single-particle to second-quantized form, return in matrix', 'Convert from single-particle to second-quantized form, return in summation (expand the matrix)', 'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)', 'Convert noninteracting Hamiltonian in real space to momentum space (lattice version)', 'Convert interacting Hamiltonian in real space to momentum space (lattice version)', 

# Prompt fill place holder

In [21]:
import json

In [22]:
pwd=os.getcwd()

In [23]:
arxiv_number=pwd.split('/')[-1]

In [32]:
with open(f'{arxiv_number}.yaml','r') as f:
    kwargs_yaml = yaml.safe_load(f)
# with open(f'{arxiv_number}.jsonl','r') as f:
#     kwargs_jsonl= [json.loads(line) for line in f]

In [25]:
def load_excerpt(sources):
    excerpt=''
    for tex, lines in sources.items():
        with open(tex,'r') as f:
            f_list=list(f)
            for line in lines:
                excerpt=excerpt+''.join(f_list[line[0]:line[1]])
    return excerpt

In [26]:
prompt_system='''I will provide you a Excerpt of physics paper, and a Template. You will need to fill the placeholders in the template using the correct information from the excerpt.
Here are conventions: 
{..} means a placeholder which you need to fill by extracting information from the excerpt.
{A|B} means you need to make a choice between A and B
[..] means optional sentence. You should decide whether to use it depending on the excerpt.
{{..}} DOES NOT mean a placeholder. You should not change the content inside double curly braces {{..}}.
`You should recall that {expression_second_nonint}.` : this sentence should be kept as is.


Finally, if you cannot figure out the placeholder, you should leave it as is.'''

In [27]:
def drop_text_after(string):
    string_new=''
    for line in string.split('\n'):
        if '===' in line:
            break
        else:
            if 'You should recall that' not in line:
                string_new='\n'.join([string_new,line])
    return string_new[1:]


In [28]:
def extractor(descriptor,message=False):
    sys_msg=[{'role': 'system', 'content': prompt_system}]
    question_prompt='\nTemplate:\n {template} \n\n Excerpt:\n {excerpt}'.format(template=drop_text_after(prompt_dict()[descriptor['task']]), excerpt=load_excerpt(descriptor['source']))
    user_msg=[{'role':'user','content':question_prompt}]
    messages = sys_msg + user_msg
    if message:
        return sys_msg[0]['content']+user_msg[0]['content']
    rs= openai.ChatCompletion.create(messages=messages, **model_params)
    response=rs['choices'][0]['message'].content
    return response


## GPT4

In [33]:
# string=''
for kwargs in tqdm(kwargs_yaml[3:]):
    string+='# '+kwargs['task']+'\n'
    response=(extractor(kwargs, message=False))
    string+=response+'\n\n'
    
    

100%|██████████| 10/10 [06:42<00:00, 40.24s/it]


In [34]:
string.count('#')

15

In [35]:
print(string)

# Construct Kinetic Hamiltonian (continuum version, single-particle)
You will be instructed to describe the kinetic term of Hamiltonian in ABC trilayer graphene in the real space in the second-quantized form.   
The degrees of freedom of the system are: six sublattice sites labeled as A, B, A', B', A'', B''.  
Express the Kinetic Hamiltonian H_0 using variables γ_0, γ_1, γ_2, γ_3, γ_N, f which are only on the diagonal terms, and arrange the basis in the order of A, B, A', B', A'', B''. [Note that the sublattice degrees of freedom is suppressed for now and will be stated later]

Use the following conventions for the symbols:  
f(k) = e^(i k_y a / sqrt(3)) * (1 + 2 e^(-i 3 k_{y} a / 2sqrt(3)) * cos(k_x a / 2)), where a = 2.46 Å.  
γ_0 = 3.12 eV, γ_1 = 0.377 eV, γ_2 = 0.01 eV, γ_3 = 0.3 eV.  
γ_N = γ_N^0 * exp(- |k - K'| / k_r), where k_r = γ_1 / ħυ_F = 0.0573 Å^-1.

# Define each term in Kinetic Hamiltonian (continuum version)
Template:
 You will be instructed to construct each term, nam

In [36]:
with open(pwd.split('/')[-1]+'_extractor.md','w') as f:
    f.write(string)